In [17]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from jugaad_data.nse import full_bhavcopy_raw, bhavcopy_save, stock_df
import pandas as pd
import random

Stupid stratergy but throwing darts works. <br>
Make a universe with the given CSV from NSE website <br>
Select 5/10/20/50 stocks from the list, allocate capital to it, set purchase price at open/day's low and on last trading date of the month sell at close/high <br>

Optional: set up charges for STT et al

In [18]:
# defining time period for back test:
from_date = date(2020, 1, 1)
to_date = date(2020, 1, 30)

In [19]:
# import the ind_niftytotalmarket_list csv file, remove all columns apart from Symbol from the same
ind_niftytotalmarket_list = pd.read_csv("ind_niftytotalmarket_list.csv")
ind_niftytotalmarket_list = ind_niftytotalmarket_list[["Symbol"]]

In [22]:
# roll a die to select x stocks from the list, after die roll, remove it from the universe and repeat until you have x stocks
num_stocks = 5
selected_stocks = []
while len(selected_stocks) < num_stocks:
    die_roll = random.randint(0, len(ind_niftytotalmarket_list) - 1)
    selected_stocks.append(ind_niftytotalmarket_list.iloc[die_roll]["Symbol"])
    ind_niftytotalmarket_list = ind_niftytotalmarket_list.drop(die_roll).reset_index(drop=True)

In [23]:
selected_stocks

['JYOTICNC', 'RELIANCE', 'GVT&D', 'INDIACEM', 'GARFIBRES']

There is a possibility that the stock did not trade during the time when we have selected for our backtest <br>
in this case, we can have a try catch block, if it is able to find data, then only try to purchase in the back test <br>
otherwise just get another stock

In [33]:
# remove the offending stock in this case, and find another stock from ind_niftytotalmarket_list
portfolio = {}
offending = []
for stock in selected_stocks:
    try:
        df = stock_df(stock, from_date, to_date)
        portfolio[stock] = df
    except:
        offending.append(stock)
        print(f"{stock} did not trade during this period")

JYOTICNC did not trade during this period


In [35]:
# remove offending from selected stocks
for stock in offending:
    selected_stocks.remove(stock)

# run the die equal to the count of length of offending stocks and add to selected stocks
while len(offending) > 0:
    die_roll = random.randint(0, len(ind_niftytotalmarket_list) - 1)
    new_stock = ind_niftytotalmarket_list.iloc[die_roll]["Symbol"]
    if new_stock not in selected_stocks:
        selected_stocks.append(new_stock)
        ind_niftytotalmarket_list = ind_niftytotalmarket_list.drop(die_roll).reset_index(drop=True)
        offending.remove(offending[0])
        try:
            df = stock_df(new_stock, from_date, to_date)
            portfolio[new_stock] = df
        except:
            offending.append(new_stock)
            selected_stocks.remove(new_stock)
            print(f"{new_stock} did not trade during this period")

In [36]:
selected_stocks

['RELIANCE', 'GVT&D', 'INDIACEM', 'GARFIBRES', 'RAMCOCEM']

In [38]:
# buy the stocks at open price on the first trading day of the month
# sell the stocks at close price on the last trading day of the month
# assume equal allocation of capital to each stock
initial_capital = 1000000
allocation_per_stock = initial_capital / len(selected_stocks)
final_capital = 0
for stock in selected_stocks:
    df = portfolio[stock]
    buy_price = df.iloc[0]["OPEN"]
    sell_price = df.iloc[-1]["CLOSE"]
    shares_bought = allocation_per_stock / buy_price
    final_capital += shares_bought * sell_price

In [40]:
# performance stats
print(f"Initial Capital: {initial_capital}")
print(f"Final Capital: {final_capital}")
print(f"Return: {(final_capital - initial_capital) / initial_capital *100:.2f}%")

Initial Capital: 1000000
Final Capital: 902918.9453775499
Return: -9.71%
